[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BioImagingUK/Training/blob/main/notebooks/OXO_IDR.ipynb)
# Cross resources example

This notebook search for a term in [IDR](https://idr.openmicroscopy.org/) and the Ontology Cross reference service [OXO](https://www.ebi.ac.uk/spot/oxo/index).


## Learning Objectives

* How to access OXO resource via its REST API.
* How to access Image resource via its REST API.
* Find terms related to a given term using OXO.
* Find images annotated using the related term.s


## Set up

Set up where to query IDR

In [4]:
INDEX_PAGE = "https://idr.openmicroscopy.org/webclient/?experimenter=-1"

# create http session
with requests.Session() as session:
    request = requests.Request('GET', INDEX_PAGE)
    prepped = session.prepare_request(request)
    response = session.send(prepped)
    if response.status_code != 200:
        response.raise_for_status()

In [5]:
import requests
import collections
from collections import defaultdict

In [6]:
SEARCH_URL = "https://idr.openmicroscopy.org/searchengine/api/v1/resources/{type}/search/"
KEY_VALUE_SEARCH = SEARCH_URL + "?key={key}&value={value}"
KEY = "Gene Symbol"

## Search for a given gene in IDR.

In [ ]:
genes = ["HGNC_5009"]

In [19]:
%%time
results = {}
for gene in genes:
    qs1 = {'type': 'image', 'key': KEY, 'value': gene}
    url = KEY_VALUE_SEARCH.format(**qs1)
    json = session.get(url).json()
    r = json['results']
    if (len(r) != 0):
      results[gene] = r['results']

if len(results) == 0:
    print("No images found")

HGNC_5009
https://idr.openmicroscopy.org/searchengine/api/v1/resources/image/search/?key=Gene Symbol&value=HGNC_5009
{'notice': '', 'query_details': {'and_filters': [{'name': 'Gene Symbol', 'operator': 'equals', 'resource': 'image', 'value': 'HGNC_5009'}], 'bookmark': [None], 'case_sensitive': None}, 'raw_elasticsearch_query': {'query': {'bool': {'must': [{'bool': {'must': {'terms': {'data_source.keyvalue': ['idr']}}}}, {'nested': {'path': 'key_values', 'query': {'bool': {'must': [{'match': {'key_values.value.keyvaluenormalize': 'HGNC_5009'}}, {'match': {'key_values.name.keynamenormalize': 'Gene Symbol'}}]}}}}]}}}, 'resource': 'image', 'results': [], 'server_query_time': '0.01'}
No images found
CPU times: user 2.87 ms, sys: 0 ns, total: 2.87 ms
Wall time: 148 ms


## Search for mapping in OXO

We use the [Ontoloty Cross Reference service](https://www.ebi.ac.uk/spot/oxo/index) to find associated terms.
We use a mapping distance of **2**.


In [20]:
URL_OXO = "https://www.ebi.ac.uk/spot/oxo/api/search"

In [22]:
headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

ids = [gene.replace("_", ":") for gene in genes]
data = {"ids" : ids, "distance" : 2}
response = requests.post(URL_OXO, headers=headers, json=data)

In [26]:
import pandas as pd

resp_json = response.json()
df_oxo = pd.DataFrame.from_records(resp_json["_embedded"]["searchResults"][0]["mappingResponseList"])
df_oxo

,curie,label,sourcePrefixes,targetPrefix,distance
0,Genatlas:HMGA2,,[Orphanet],Genatlas,2
1,Reactome:P52926,,[Orphanet],Reactome,2
2,OMIM:600698,,[Orphanet],OMIM,2
3,Ensembl:ENSG00000149948,,[Orphanet],Ensembl,2
4,Orphanet:248487,high mobility group AT-hook 2,[Orphanet],Orphanet,1


### Parse the mapping from OXO

In [28]:
queries = resp_json["_embedded"]["searchResults"]
related_genes = []
for i in range(len(queries)):
    mappings = queries[i]["mappingResponseList"]
    for j in range(len(mappings)):
        curie = mappings[j]["curie"]
        related_genes.append(curie.split(":")[1])
print(related_genes)

['HMGA2', 'P52926', '600698', 'ENSG00000149948', '248487']


### Search for the genes in IDR

Search for the list of related genes found using OXO.

In [30]:
results = {}
for gene in related_genes:
    qs1 = {'type': 'image', 'key': KEY, 'value': gene}
    url = KEY_VALUE_SEARCH.format(**qs1)
    json = session.get(url).json()
    r = json['results']
    if (len(r) != 0):
      results[gene] = r['results']
print(results)

{'HMGA2': [{'data_source': 'idr', 'dataset_id': None, 'dataset_name': None, 'description': None, 'doc_type': 'image_keyvalue_pair_metadata', 'experiment': None, 'group_id': 3, 'id': 633612, 'image_size': None, 'key_values': [{'index': 0.0, 'name': 'Cell Line', 'value': 'HeLa'}, {'index': 0.0, 'name': 'Gene Annotation Comments', 'value': 'Ensembl release 81 gene symbol added by IDR curator'}, {'index': 0.0, 'name': 'Gene Identifier', 'value': 'ENSG00000149948'}, {'index': 0.0, 'name': 'Organism', 'value': 'Homo sapiens'}, {'index': 0.0, 'name': 'Quality Control', 'value': 'pass'}, {'index': 0.0, 'name': 'Sense Sequence', 'value': 'GCCCUCUCCUAAGAGACCCtt'}, {'index': 0.0, 'name': 'siRNA Identifier', 'value': '243318'}, {'index': 1.0, 'name': 'Analysis Gene Annotation Build', 'value': 'GRCh37, Ensembl release 61, Feb 2011, Gene Symbols from Ensembl release 61 or 81'}, {'index': 1.0, 'name': 'Antisense Sequence', 'value': 'GGGUCUCUUAGGAGAGGGCtc'}, {'index': 1.0, 'name': 'Channels', 'value':

Found the phenotype associated to the gene.

In [41]:
REFERENCE_KEY = "Phenotype"

phenotypes = {}
for k in results:
    images = results[k]
    result_images = []
    for image in images:
        values = image["key_values"]
        for v in values:
            name = v["name"]
            value = v['value']
            if name == REFERENCE_KEY:
                if value not in phenotypes:
                    phenotypes[value] = []
                phenotypes[value].append(image["id"])
print(phenotypes)

{'Increased or decreased total RNA production rate': [12608478, 12608479, 12608480, 12608481, 12608482, 12608483, 12608484, 12608485, 12608486]}


## Display the images

Prepare URLs for the display of images associated to the genes.

In [45]:
# URLs to retrieve the thumbnails and link to the images in IDR
BASE_URL = "https://idr.openmicroscopy.org/webclient"
IMAGE_DATA_URL = BASE_URL + "/render_thumbnail/{id}"
LINK_URL = BASE_URL + "/?show=image-{id}"

In [46]:
# Display the images
from ipywidgets import AppLayout, widgets

table_widget = widgets.HTML("")

html = "<table>"
for phenotype in phenotypes:
    images = phenotypes[phenotype]
    print(images)
    if len(images) > 0:
        html += '<tr><td><h2>Phenotype: '+phenotype+'</h2></td></tr><tr>'
        for id in images:
            qs = {'id': id}
            url = IMAGE_DATA_URL.format(**qs)
            url_link = LINK_URL.format(**qs)
            html += '<td><a href="'+url_link+'" target="_blank"><img src="'+url+'"/></a></td>'
        html += "</tr>"
html += "</table>"

table_widget.value = html
AppLayout(header=None,
          left_sidebar=None,
          center=table_widget,
          right_sidebar=None,
          footer=None)

[12608478, 12608479, 12608480, 12608481, 12608482, 12608483, 12608484, 12608485, 12608486]


AppLayout(children=(HTML(value='<table><tr><td><h2>Phenotype: Increased or decreased total RNA production rate…